# Módulo 4 — IA Generativa para TI (BCN / SISRED)
## Notebook: Prompt Engineering aplicado a un caso SISRED + Visión (imagen)

**Objetivo del notebook (práctico y realista):**
1. Aplicar *Prompt Engineering* (Rol → Contexto → Tarea → Restricciones → Formato).
2. Construir un *asistente de triage* para SISRED: clasificar solicitudes y proponer acciones.
3. Probar una celda multimodal: **subir una imagen** + prompt: *“defíneme qué está pasando en la imagen”*.


## 0) Preparación (1 minuto)

### Requisitos
- Python 3.9+
- `pip install openai pillow numpy pandas`

### API Key
Define tu variable de entorno en tu terminal antes de ejecutar:
- Windows (PowerShell): `setx OPENAI_API_KEY "TU_KEY"`
- macOS/Linux: `export OPENAI_API_KEY="TU_KEY"`


## 0) Preparación (1 minuto)

### Requisitos
- `pip install openai pillow numpy pandas`

### API Key
Define tu variable de entorno en tu terminal antes de ejecutar:
- Windows (PowerShell): `setx OPENAI_API_KEY "TU_KEY"`
- macOS/Linux: `export OPENAI_API_KEY="TU_KEY"`


In [15]:

%pip install -q openai

import os
import json
import base64
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image

from openai import OpenAI

client = OpenAI()

# Modelos (ajusta si tu org lo requiere)
MODEL_TEXT = "gpt-4.1-mini"
MODEL_VISION = "gpt-4.1-mini"

print("OK: librerías cargadas. API key detectada:", bool(os.getenv("OPENAI_API_KEY")))


Note: you may need to restart the kernel to use updated packages.


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable